This demo shows how to perform latent gender prediction from trained MIMIC-III models.

In [ ]:
import numpy as np
from torch import nn
import torch
from utils.vit import TransformerEncoder
from utils.data_loaders import *
from utils.utils import *
from easydict import EasyDict as edict

In [ ]:
train_loader,val_loader,test_loader=get_loaders_IHM(sampler=False)

In [ ]:
## load a stored model, could be LSTM, TF or TCN
device=get_device()
print(device)
model=torch.load('./RNN')
model.to(device)



In [ ]:
loss_fn = nn.BCELoss().to(device)
loss,auc=prediction_binary(model,test_loader,loss_fn,device) 
auc

In [ ]:
from tqdm import tqdm
EMB_train=[]
for i,data in tqdm(enumerate(train_loader)):
    
    model.eval()
    inputs,label=data
    inputs=inputs.to(torch.float32).to(device)
    label=label.to(torch.float32).to(device)
 
    e=model.emb(inputs)
    EMB_train.append(e.detach().cpu())

EMB_val=[]
for i,data in tqdm(enumerate(val_loader)):
    
    model.eval()
    inputs,label=data
    inputs=inputs.to(torch.float32).to(device)
    label=label.to(torch.float32).to(device)
 
    e=model.emb(inputs)
    EMB_val.append(e.detach().cpu())
    
EMB_test=[]   

for i,data in tqdm(enumerate(test_loader)):
    
    model.eval()
    inputs,label=data
    inputs=inputs.to(torch.float32).to(device)
    label=label.to(torch.float32).to(device)
 
    e=model.emb(inputs)
    EMB_test.append(e.detach().cpu())
        

In [ ]:
Emb_train=np.vstack(EMB_train)
Emb_test=np.vstack(EMB_test)
Emb_val=np.vstack(EMB_val)

In [ ]:
import pandas as pd

## We have these files containing gender info for mimic patients
A=pd.read_csv('Train_withAgeandSex.csv')
A['Gender'].replace(['F','M'],[0,1],inplace=True)
Sex_train=A['Gender']

A=pd.read_csv('Val_withAgeandSex.csv')
A['Gender'].replace(['F','M'],[0,1],inplace=True)
Sex_val=A['Gender']


A=pd.read_csv('Test_withAgeandSex.csv')
A['Gender'].replace(['F','M'],[0,1],inplace=True)
Sex_test=A['Gender']



In [ ]:
from utils.Recurrent_Models import Emb_pheno
from sklearn.metrics import roc_auc_score,roc_curve


emb_dim=256 ## change it as per the model emb dim
out=1

model=Emb_pheno(emb_dim,out,device)
model.to(device)


train_dataset= TensorDataset(torch.tensor(Emb_train),torch.tensor(Sex_train))
train_loader = DataLoader(train_dataset, batch_size=256,shuffle=True)    

val_dataset= TensorDataset(torch.tensor(Emb_val),torch.tensor(Sex_val))
val_loader = DataLoader(val_dataset, batch_size=256,shuffle=False)   

test_dataset= TensorDataset(torch.tensor(Emb_test),torch.tensor(Sex_test))
test_loader = DataLoader(test_dataset, batch_size=256,shuffle=False) 


In [ ]:
opt = torch.optim.Adam(params=model.parameters(),lr=0.001)
loss_fn = nn.BCELoss().to(device)
best=0

In [ ]:
for epoch in range(0,500):
    train_loss=0
    for i,data in enumerate(train_loader):


        model.train()
        opt.zero_grad()
      

        inputs,label=data
        inputs=inputs.to(torch.float32).to(device)
        label=label.to(torch.float32).to(device)
 
        pred=model(inputs)
        loss=loss_fn(pred[:,0],label)

        loss.backward()
 
        opt.step()     
        train_loss=train_loss+loss.detach().cpu()
         
      
    val_loss,auc=prediction_binary(model,val_loader,loss_fn,device)

    if auc>best:
       best=auc
       torch.save(model,'./embedding_sex')

    print('Epoch : {:.1f} Train Loss {:.4f} Val Loss {:.4f} Val AUROC {:.4f}'.format(epoch,train_loss/len(train_loader),val_loss,auc))  
    

In [ ]:
model=torch.load('./embedding_sex')
loss,auc=prediction_binary(model,test_loader,loss_fn,device) 
print(auc)